## Fetch & Filter Workunit Extent Spatial Metadata (WESM)

**Goal**
- Fetch WESM data (.cvs or .gpkg) from hosted s3 without downloading locally
- Filter for QL 1 & QL 0 projects in Minnesota
- Write the filtered data to an interim geopackage and geoparquet

### Asset URLs

In [1]:
wesm_gpkg_url = (
    "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/metadata/WESM.gpkg"
)
wesm_csv_url = (
    "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/metadata/WESM.csv"
)
opr_tesm_gpkg_url = "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/OPR/FullExtentSpatialMetadata/OPR_TESM.gpkg"

### Download Functions

In [2]:
from pathlib import Path
import requests


def download_usgs_asset(url: str, output_dir: Path) -> Path:
    filename = url.split("/")[-1]
    output_file = output_dir / filename
    r = requests.get(url, stream=True)
    with open(output_file, "wb") as f:
        f.write(r.content)
    return output_file

In [3]:
download_usgs_asset(
    url=wesm_csv_url, output_dir=Path("../data/external/usgs/").resolve()
)

PosixPath('/home/dpower/projects/geospatial/culvert-vision/data/external/usgs/WESM.csv')

In [4]:
from pathlib import Path
import requests
from zipfile import ZipFile, ZIP_DEFLATED


def download_usgs_asset_to_zip(
    url: str, output_dir: Path, compression: int = ZIP_DEFLATED
) -> Path:
    filename = url.split("/")[-1]
    output_file = output_dir / f"{filename}.zip"
    r = requests.get(url, stream=True)
    with ZipFile(output_file, "w", compression=compression) as f:
        f.writestr(zinfo_or_arcname=filename, data=r.content)
    return output_file

In [5]:
download_usgs_asset_to_zip(
    url=wesm_csv_url, output_dir=Path("../data/external/usgs/").resolve()
)

PosixPath('/home/dpower/projects/geospatial/culvert-vision/data/external/usgs/WESM.csv.zip')

In [6]:
from datetime import datetime
from email.utils import parsedate_to_datetime


def fetch_last_modified_datetime(url: str) -> datetime:
    r = requests.head(url)
    return parsedate_to_datetime(r.headers["Last-Modified"])

In [7]:
fetch_last_modified_datetime(wesm_csv_url)

datetime.datetime(2023, 5, 27, 17, 15, 5, tzinfo=datetime.timezone.utc)

### Filtering WESM.csv

In [8]:
import pandera
from pandera.typing.pandas import Series as PaSeries


class FilteredWESM(pandera.DataFrameModel):
    workunit: PaSeries[str]
    workunit_id: PaSeries[int]
    ql: PaSeries[str]
    horiz_crs: PaSeries[int]
    dem_gsd_meters: PaSeries[float]
    lpc_link: PaSeries[str]
    sourcedem_link: PaSeries[str]
    metadata_link: PaSeries[str]

    class Config:
        strict = "filter"
        coerce = True

In [9]:
from pathlib import Path
import pandas as pd
from pandas import DataFrame


def _filter_rows_by_workunit(df: DataFrame, workunit_contains: str) -> DataFrame:
    return df.loc[df["workunit"].str.contains(workunit_contains)]


def _filter_rows_by_quality_level(df: DataFrame, quality_level: int) -> DataFrame:
    return df.query(f"ql == 'QL {quality_level}'")


def get_filtered_wesm(
    wesm_csv: Path, workunit_contains: str = "MN_", quality_levels: list[int] = [0, 1]
) -> DataFrame:
    df = pd.read_csv(wesm_csv)
    df = _filter_rows_by_workunit(df=df, workunit_contains=workunit_contains)
    concatinated = pd.concat(
        [
            _filter_rows_by_quality_level(df=df, quality_level=ql)
            for ql in quality_levels
        ]
    )
    return FilteredWESM.validate(concatinated)

In [10]:
import pandas as pd

wesm_csv = download_usgs_asset(
    wesm_csv_url, output_dir=Path("../data/external/usgs/").resolve()
)

df = pd.read_csv(wesm_csv)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2769 entries, 0 to 2768
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   workunit            2769 non-null   object 
 1   workunit_id         2769 non-null   int64  
 2   project             2769 non-null   object 
 3   project_id          2769 non-null   int64  
 4   collect_start       2769 non-null   object 
 5   collect_end         2769 non-null   object 
 6   ql                  2768 non-null   object 
 7   spec                2768 non-null   object 
 8   p_method            2768 non-null   object 
 9   dem_gsd_meters      2533 non-null   float64
 10  horiz_crs           2769 non-null   object 
 11  vert_crs            2769 non-null   object 
 12  geoid               2766 non-null   object 
 13  lpc_pub_date        2320 non-null   object 
 14  lpc_category        2766 non-null   object 
 15  lpc_reason          2766 non-null   object 
 16  source

In [12]:
mn_wesm = get_filtered_wesm(wesm_csv)
assert mn_wesm.shape[0] == 13

In [13]:
mn_workunits = mn_wesm["workunit"].to_list()

### Fetch EPT Catalog Items

In [14]:
from pystac_client import Client
from functools import partial


STAC_CATALOG_URL = "https://usgs-lidar-stac.s3-us-west-2.amazonaws.com/ept/catalog.json"


def _get_item_link(workunit: str, client: Client) -> str | None:
    item_links = [link for link in client.get_item_links() if workunit in link.href]
    return item_links[0].href if len(item_links) > 0 else None


def _add_ept_item_link_column(df: DataFrame) -> DataFrame:
    client = Client.open(STAC_CATALOG_URL)
    fetch_item_links = partial(_get_item_link, client=client)
    item_links = df["workunit"].apply(fetch_item_links)
    return df.assign(ept_item_link=item_links)

In [15]:
mn_wesm_with_ept = _add_ept_item_link_column(mn_wesm).dropna()

In [16]:
mn_wesm_with_ept["workunit_id"].to_list()

[228991,
 187243,
 300017,
 197389,
 228969,
 218907,
 222535,
 228424,
 228972,
 223443]

### Read tiles to geoparquet

In [17]:
rows = [row for row in mn_wesm_with_ept.itertuples(index=False, name="Wesm")]
rows

[Wesm(workunit='MN_SEDriftless_5_2021', workunit_id=228991, ql='QL 0', dem_gsd_meters=0.5, horiz_crs=6344, lpc_link='https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/MN_SE_Driftless_2021_B21/MN_SEDriftless_5_2021', sourcedem_link='http://prd-tnm.s3.amazonaws.com/index.html?prefix=StagedProducts/Elevation/OPR/Projects/MN_SE_Driftless_2021_B21/MN_SEDriftless_5_2021', metadata_link='http://prd-tnm.s3.amazonaws.com/index.html?prefix=StagedProducts/Elevation/metadata/MN_SE_Driftless_2021_B21/MN_SEDriftless_5_2021', ept_item_link='https://s3-us-west-2.amazonaws.com/usgs-lidar-stac/ept/MN_SEDriftless_5_2021.json'),
 Wesm(workunit='MN_LakeCounty_2018', workunit_id=187243, ql='QL 1', dem_gsd_meters=0.3, horiz_crs=6344, lpc_link='https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/MN_LakeCounty_2018_C20/MN_LakeCounty_2018', sourcedem_link='http://prd-tnm.s3.amazonaws.com/index.html?prefix=StagedProducts/Elevation/OPR/Projects/MN_LakeCounty_2018_C2

In [18]:
interim_tile_index_dir = Path("../data/interim/tile_index/").resolve()
opr_tesm_parquet = Path("../data/interim/opr_tesm.parquet").resolve()

In [21]:
import geopandas
from geopandas import GeoDataFrame


for row in rows:
    gdf: GeoDataFrame = geopandas.read_parquet(
        opr_tesm_parquet, filters=[("workunit_id", "=", row.workunit_id)]
    )
    output_file = interim_tile_index_dir / f"{row.workunit}.parquet"
    gdf.to_crs(epsg=row.horiz_crs).to_parquet(output_file)
    print(f"Wrote parquet for {row.workunit}")
    del gdf

Wrote parquet for MN_SEDriftless_5_2021
Wrote parquet for MN_LakeCounty_2018
Wrote parquet for MN_RainyLake_2_2020
Wrote parquet for MN_RainyLake_1_2020
Wrote parquet for MN_SEDriftless_1_2021
Wrote parquet for MN_LakeSuperior_1_2021
Wrote parquet for MN_SEDriftless_2_2021
Wrote parquet for MN_LakeSuperior_2_2021
Wrote parquet for MN_SEDriftless_3_2021
Wrote parquet for MN_MORiverBigSioux_1_B21


In [24]:
parquet = Path("../data/interim/tile_index/MN_RainyLake_1_2020.parquet").resolve()
gdf: GeoDataFrame = geopandas.read_parquet(parquet)
assert gdf.crs.to_epsg() == 6344

In [25]:
gdf.head()

,fid,tile_id,project,project_id,workunit_id,geometry
0,386813,15TWN553291,MN_RainyLake_2020_B20,197392,197389,"MULTIPOLYGON (((554000.056 5292000.015, 553000..."
1,386814,15TXN607259,MN_RainyLake_2020_B20,197392,197389,"MULTIPOLYGON (((607999.990 5259999.847, 606999..."
2,386815,15UWP564321,MN_RainyLake_2020_B20,197392,197389,"MULTIPOLYGON (((565000.256 5321999.973, 564000..."
3,386816,15TWN528284,MN_RainyLake_2020_B20,197392,197389,"MULTIPOLYGON (((529000.168 5284999.982, 528000..."
4,386817,15TXN631290,MN_RainyLake_2020_B20,197392,197389,"MULTIPOLYGON (((631999.933 5290999.773, 630999..."
